In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('/content/Train.csv')
df_test = pd.read_csv('/content/Test.csv')

<b>Let's fix the abnormal punctuations

In [ ]:
# punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
punct = "/-?!.,#$%\()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
def fix_punt(sentence,punct):
    
    for p in punct:
#         sentence = sentence.replace(p, f' {p} ')
        sentence = sentence.replace(p, f'')
    return sentence

In [ ]:
df['new_questions'] = df['question'].apply(lambda x: fix_punt(x,punct))
df['new_answers'] = df['answer_text'].apply(lambda x: fix_punt(x,punct))

__Also, let's combine all the text including distractors, answers and questions, reason being:__
 * I will increase the length of text corpus, thus we'll have more data to train the model.
 * The combined corpus will help model to understand the context of distractors with respect to provided questions and answers, thus model might generate better text when question and answer are provided to it as an input later.

In [ ]:
df["new"] = df["new_questions"].map(str) + " " + df["new_answers"]

In [ ]:
df['dist'] = df['distractor'].apply(lambda x: fix_punt(x,punct+"'"))
a = df['dist'][1]

In [ ]:
df['new_corpus'] = df["new"].map(str) + " " + df["dist"]

In [ ]:
import re
df['new_corpus'] = df['new_corpus'].apply(lambda x: re.sub(' +', ' ',x).strip())

In [9]:
df.head()

,question,answer_text,distractor,new_questions,new_answers,new,dist,new_corpus
0,Meals can be served,in rooms at 9:00 p. m.,"'outside the room at 3:00 p. m.', 'in the dini...",Meals can be served,in rooms at 900 p m,Meals can be served in rooms at 900 p m,outside the room at 300 p m in the dining roo...,Meals can be served in rooms at 900 p m outsid...
1,It can be inferred from the passage that,The local government can deal with the problem...,"'If some tragedies occur again ', ' relevant d...",It can be inferred from the passage that,The local government can deal with the problem...,It can be inferred from the passage that The l...,If some tragedies occur again relevant depar...,It can be inferred from the passage that The l...
2,The author called Tommy 's parents in order to,help them realize their influence on Tommy,"'blame Tommy for his failing grades', 'blame T...",The author called Tommy 's parents in order to,help them realize their influence on Tommy,The author called Tommy 's parents in order to...,blame Tommy for his failing grades blame Tommy...,The author called Tommy 's parents in order to...
3,It can be inferred from the passage that,the writer is not very willing to use idioms,'idioms are the most important part in a langu...,It can be inferred from the passage that,the writer is not very willing to use idioms,It can be inferred from the passage that the w...,idioms are the most important part in a langua...,It can be inferred from the passage that the w...
4,How can we deal with snake wounds according to...,Stay calm and do n't move .,'Cut the wound and suck the poison out .',How can we deal with snake wounds according to...,Stay calm and do n't move,How can we deal with snake wounds according to...,Cut the wound and suck the poison out,How can we deal with snake wounds according to...


In [ ]:
df= df.drop(labels=['new_questions','new_answers','dist'], axis=1)

In [11]:
df['new_corpus'].to_csv('data.csv', index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


<b>We are going to use pretrained [GPT-2 model (small)](https://openai.com/blog/better-language-models/) by finetunining it on our dataset for generating distractors.

In [12]:
!pip install -q gpt-2-simple

     |████████████████████████████████| 655kB 16.7MB/s eta 0:00:01


In [ ]:
import gpt_2_simple as gpt2

In [14]:
gpt2.download_gpt2()

Fetching checkpoint: 1.05Mit [00:00, 521Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 99.4Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 326Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:04, 107Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 453Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 198Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 209Mit/s]                                                       


__Finetuning the model on Google Colab using GPU so as to speed up the process__

In [15]:
!nvidia-smi

Mon Oct  7 05:10:25 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [16]:
gpt2.mount_gdrive()
# gpt2.copy_checkpoint_from_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [17]:
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
             dataset = '/content/data.csv',
             model_name='124M',
             steps=1000,
             restore_from='fresh',
             run_name='run1',
             print_every=10,
             sample_every=200,
             save_every=500)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


100%|██████████| 1/1 [00:00<00:00, 11.52it/s]

Loading dataset...


dataset has 1584669 tokens
Training...
[10 | 50.50] loss=2.82 avg=2.82
[20 | 93.71] loss=2.62 avg=2.72
[30 | 136.90] loss=2.55 avg=2.66
[40 | 179.96] loss=2.58 avg=2.64
[50 | 223.05] loss=2.54 avg=2.62
[60 | 266.10] loss=2.47 avg=2.59
[70 | 309.32] loss=2.47 avg=2.58
[80 | 352.53] loss=2.41 avg=2.55
[90 | 395.74] loss=2.52 avg=2.55
[100 | 438.89] loss=2.51 avg=2.54
[110 | 482.05] loss=2.36 avg=2.53
[120 | 525.25] loss=2.45 avg=2.52
[130 | 568.36] loss=2.42 avg=2.51
[140 | 611.56] loss=2.46 avg=2.51
[150 | 654.76] loss=2.34 avg=2.50
[160 | 697.96] loss=2.45 avg=2.49
[170 | 741.17] loss=2.28 avg=2.48
[180 | 784.38] loss=2.34 avg=2.47
[190 | 827.58] loss=2.27 avg=2.46
[200 | 870.74] loss=2.40 avg=2.46
======== SAMPLE 1 ========
text>The author did n't think of his first name at first because most people would have a mistake of grammar if their first name were changed it was too difficult for his parents to remember the correct spelling<|endoftext|>
<|startoftext|>Which of the following st

It took around 1 hour to fine tune the model on Google Colb using Nvidia Tesla K80 GPU, let's save the model first

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

In [18]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1000
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


In [19]:
seed = str(df['question'][0])
print(seed)

Meals can be served


In [21]:
gpt2.generate(sess,
              length= len(seed.split()) + 15,
              prefix=seed,
              temperature=0.7,
              top_k = 5,
              top_p=0.9,
              include_prefix = False,
              truncate ='<|endoftext|>',
              return_as_list=True,
              nsamples=3,
              batch_size=3
              )


[' in the morning at the cafe',
 ' only at the cafe Cafe della Suave La Forum',
 ' in 3 parts']

__We can see the GPT-2 Model is Generating Text which is making sense in context with the quesion text__

__Let's generate Distractors for test set__

In [22]:
df_test.head()

,question,answer_text
0,What 'S the main idea of the text ?,The lack of career -- based courses in US high...
1,"In the summer high season , Finland does nt se...",the sun is out at night
2,If you want to apply for Chinese Business Inte...,have to get confirmed at least twice
3,"That afternoon , the boy 's clothes were dry b...",nobody made room for him in the water .
4,Which of the following statements is NOT true ?,There are twelve countries in the World Wildli...


In [23]:
seed2 = str(df_test['question'][1]+" "+df_test['answer_text'][1])
print(seed2)

In the summer high season , Finland does nt seem to sleep because the sun is out at night


In [24]:
gpt2.generate(sess,
              length=15,
              prefix=seed2,
              temperature=0.7,
              top_k = 5,
              top_p=0.9,
              include_prefix = False,
              truncate ='<|endoftext|>',
              return_as_list=True,
              nsamples=3,
              batch_size=3
              )


['In the summer high season , Finland does nt seem to sleep because the sun is out at night it is in winter when the sun is shining<|endoftext|',
 'In the summer high season , Finland does nt seem to sleep because the sun is out at night the temperature is low in winter people have to stay up late in order to',
 '']

In [ ]:
df_test['new'] = df_test['question'].apply(lambda x: fix_punt(x,punct)) +" "+df_test['answer_text'].apply(lambda x: fix_punt(x,punct))

In [ ]:
def gen_output(data):
  return ", ".join(gpt2.generate(sess,
                                length=15,
                                prefix=str(data),
                                temperature=0.7,
                                top_k = 5,
                                top_p=0.9,
                                include_prefix = False,
                                truncate ='<|endoftext|>',
                                return_as_list=True,
                                nsamples=3,
                                batch_size=3
                                )
                  )

In [ ]:
df_test['distractor'] = df_test['new'].apply(lambda x: gen_output(x))


In [ ]:
df_test.drop(labels=['new'], axis=1)

In [ ]:
df_test.head()

In [ ]:
df_test.to_csv('submission.csv', index=False)


In [ ]:
!cp /content/submission.csv /content/drive/My\ Drive/